task name: Resolve LogLossPerClass Task
task description: Implement the LogLossPerClass example using the provided C# code and save the result to an output file.

In [ ]:
#r "nuget:Microsoft.ML,1.5.5"

In [ ]:
using System;
using System.Collections.Generic;
using System.Linq;
using Microsoft.ML;
using Microsoft.ML.Data;

In [ ]:
var mlContext = new MLContext(seed: 0);

In [ ]:
// Generate data points with random features and labels 1 to 9.
private static IEnumerable<DataPoint> GenerateRandomDataPoints(int count,
    int seed = 0)
{
    var random = new Random(seed);
    float randomFloat() => (float)(random.NextDouble() - 0.5);
    for (int i = 0; i < count; i++)
    {
        // Generate Labels that are integers 1, 2 or 3
        var label = random.Next(1, 10);
        yield return new DataPoint
        {
            Label = (uint)label,
            // Create random features that are correlated with the label.
            // The feature values are slightly increased by adding a
            // constant multiple of label.
            Features = Enumerable.Repeat(label, 20)
                .Select(x => randomFloat() + label * 0.2f).ToArray()

        };
    }
}

// Example with label and 20 feature values. A data set is a collection of
// such examples.
private class DataPoint
{
    public uint Label { get; set; }
    [VectorType(20)]
    public float[] Features { get; set; }
}

var dataPoints = GenerateRandomDataPoints(1000);
var trainingData = mlContext.Data.LoadFromEnumerable(dataPoints);

In [ ]:
var pipeline =
    mlContext.Transforms.Conversion.MapValueToKey(nameof(DataPoint.Label));

var trainer = mlContext.MulticlassClassification.Trainers.SdcaNonCalibrated();

var pipelineWithTrainer = pipeline.Append(trainer);
var model = pipelineWithTrainer.Fit(trainingData);

In [ ]:
var testData = mlContext.Data.LoadFromEnumerable(GenerateRandomDataPoints(500, seed: 123));
var transformedTestData = model.Transform(testData);

In [ ]:
var metrics = mlContext.MulticlassClassification.Evaluate(transformedTestData);

In [ ]:
VBuffer<uint> keys = default;
transformedTestData.Schema["PredictedLabel"].GetKeyValues(ref keys);
var originalLabels = keys.DenseValues().ToArray();
for (var i = 0; i < originalLabels.Length; i++)
    Console.WriteLine("LogLoss for label " + originalLabels[i] + ": " + metrics.PerClassLogLoss[i].ToString("F4"));